In [1]:
from Hamil_search import *
from simulation import *
import numpy as np

# Test 1D TFIM with OBC

In [59]:
lambdaPen = 1024
blockNum = 2
n = 4
size = n*blockNum

Hpen_block = []
Htar_block = []
for i in range(blockNum):
    Htar_block.append((1, f'Z{2*i}*Z{2*i+1}'))
    Htar_block.append((1, f'Z{2*i}'))
    Htar_block.append((1, f'Z{2*i+1}'))
    Htar_block.append((1, f'X{2*i}'))
    Htar_block.append((1, f'X{2*i+1}'))
    Hpen_block += [(1, f'X{4*i}*X{4*i+1}'), (3, f'Z{4*i}*Z{4*i+1}'), (1, f'X{4*i+2}*X{4*i+3}'), (3, f'Z{4*i+2}*Z{4*i+3}')]
for i in range(blockNum-1):
    Htar_block.append((1, f'Z{2*i+1}*Z{2*i+2}'))
Henc_block = EncodeTar(Htar_block, blockNum, lambdaPen)
U = getU(n, blockNum)

P = U @ U.conj().T
Q = np.identity(P.shape[0]) - P
Henc = blocks2Mat(size, Henc_block)
Hpen = blocks2Mat(size, Hpen_block)
Htar = blocks2Mat(size//2, Htar_block)
HpenInverse = np.linalg.pinv(Hpen)

print(checkSame(P@Hpen, np.zeros(P.shape)))
print(checkSame(P@Hpen@Q, np.zeros(P.shape))) 
print(checkSame(P@Henc@P - (P@Henc@Q@HpenInverse@Q@Henc@P / lambdaPen), U@Htar@U.conj().T))
Hsim = lambdaPen * Hpen + Henc

True
True
True


In [60]:
np.random.seed(42)
epsilons = np.random.uniform(-0.1,0.1,size*3)
# epsilons = np.zeros(size*3)
V = [PauliTerm(size, f'X{i}', epsilons[i]) for i in range(size)] + [PauliTerm(size, f'Z{i}', epsilons[i+size]) for i in range(size)] + [PauliTerm(size, f'Y{i}', epsilons[i+size]) for i in range(size)]
V = sum([p.value() for p in V])

print(checkSame(P@V@P, np.zeros(P.shape)))

True


In [61]:
# ---- previous version ----
t = 0.1
Hleft = expm(-1j * t * (Hsim + V))
Hright = U @ expm(-1j * t * Htar) @ U.conj().T
err = np.linalg.norm((Hleft - Hright)@P, ord = 2)
print(f"t={t}, err={err}")

t=0.1, err=0.03174346824197408


In [41]:
# @jit(nopython=True)
# def exp_minus_iHt(H, t):
#     e, u = np.linalg.eigh(H)
#     e = np.expand_dims(e, axis=1)
#     A = u.conj().T
#     A = np.exp(-1j*e*t) * A
#     A = u @ A
#     return A

# @jit(nopython=True)
# def exp_minus_iHt_mult_B(H, t, B):
#     e, u = np.linalg.eigh(H)
#     e = np.expand_dims(e, axis=1)
#     A = u.conj().T @ B
#     A = np.exp(-1j*e*t) * A
#     A = u @ A
#     return A

# @jit(nopython=True)
# def time_evol_err(t, Hsim, V, Htar, U):
#     Hleft = exp_minus_iHt_mult_B(Hsim + V, t, U)
#     Hright = U @ exp_minus_iHt(Htar, t)
#     err = np.linalg.norm(Hleft - Hright, ord = 2)
#     return err

In [62]:
def exp_minus_iHt(H, t):
    """
    e^{-iHt}
    """
    e, u = np.linalg.eigh(H)
    e = np.expand_dims(e, axis=1)
    A = u.conj().T
    A = np.exp(-1j*e*t) * A
    A = u @ A
    return A

def exp_minus_iHt_mult_B(H, t, B):
    """
    e^{-iHt} B
    """
    e, u = np.linalg.eigh(H)
    e = np.expand_dims(e, axis=1)
    A = u.conj().T @ B
    A = np.exp(-1j*e*t) * A
    A = u @ A
    return A

def time_evol_err(t, Hsim, V, Htar, U):
    Hleft = exp_minus_iHt_mult_B(Hsim + V, t, U)
    Hright = U @ exp_minus_iHt(Htar, t)
    err = np.linalg.norm(Hleft - Hright, ord = 2)
    return err

In [63]:
time_evol_err(0.1, Hsim, V, Htar, U)

0.031743468241971176

In [64]:
t = 0.1
Hleft = expm(-1j * t * (Hsim + V)) @ U
Hright = U @ expm(-1j * t * Htar)
np.linalg.norm(Hleft - Hright, ord = 2)

0.03174346824197411

In [65]:
Q0 = HpenInverse @ Hpen
P0 = np.identity(Q0.shape[0]) - Q0
Heff_2nd_order = P0@Henc@P0 - (P0@Henc@Q0@HpenInverse@Q0@Henc@P0 / lambdaPen)

In [66]:
print(checkSame(P @ Heff_2nd_order @ P, U@Htar@U.conj().T))

True


In [67]:
print(checkSame(P @ Heff_2nd_order @ (P0-P), np.zeros(P.shape)))

True


# Test 4-site 1D TFIM with PBC

In [5]:
lamb = 1024
blockNum = 2
size = 4 * blockNum

Hpen_block = []
for i in range(blockNum):
    Hpen_block += [(1, f'X{4*i}*X{4*i+1}'), (3, f'Z{4*i}*Z{4*i+1}'), (1, f'X{4*i+2}*X{4*i+3}'), (3, f'Z{4*i+2}*Z{4*i+3}')]
Hpen = blocks2Mat(size, Hpen_block)
HpenInverse = np.linalg.pinv(Hpen)
Q0 = HpenInverse @ Hpen
P0 = np.identity(Q0.shape[0]) - Q0

Htar_block = []
Henc_block = []
Htar_block.append((1, 'Z0*Z1'))
Henc_block.append((1, 'Z1*Z2'))
Htar_block.append((1, 'Z2*Z3'))
Henc_block.append((1, 'Z5*Z6'))
Htar_block.append((1, 'Z1*Z2'))
Henc_block += [(1, f'Z4*Z5'), ((8*lamb/3)**0.5, f'Z1*X6+Z3*X6')]
Htar_block.append((1, 'Z0*Z3'))
Henc_block += [(1, f'Z0*Z1'), ((8*lamb/3)**0.5, f'X2*Z5+X2*Z7')]
Htar = blocks2Mat(size//2, Htar_block)
Henc = blocks2Mat(size, Henc_block)

Hsim = lamb * Hpen + Henc
Heff_2nd_order = P0 @ Henc @ P0 - (P0 @ Henc @ Q0 @ HpenInverse @ Q0 @ Henc @ P0 / lamb)

Uenc = getU(4, blockNum)
Penc = Uenc @ Uenc.conj().T

print(checkSame(Penc @ Heff_2nd_order @ Penc, Uenc @ Htar @ Uenc.conj().T))
print(checkSame(Penc @ Heff_2nd_order @ (P0-Penc), np.zeros(Penc.shape)))

True
True


# Test 3x2 2D TFIM with OBC

In [6]:
lamb = 1024
blockNum = 3
size = 4 * blockNum

Hpen_block = []
for i in range(blockNum):
    Hpen_block += [(1, f'X{4*i}*X{4*i+1}'), (3, f'Z{4*i}*Z{4*i+1}'), (1, f'X{4*i+2}*X{4*i+3}'), (3, f'Z{4*i+2}*Z{4*i+3}')]
Hpen = blocks2Mat(size, Hpen_block)
HpenInverse = np.linalg.pinv(Hpen)
Q0 = HpenInverse @ Hpen
P0 = np.identity(Q0.shape[0]) - Q0

Htar_block = []
Henc_block = []
Htar_block.append((1, 'Z0*Z1'))
Henc_block.append((1, 'Z1*Z2'))
Htar_block.append((1, 'Z2*Z3'))
Henc_block.append((1, 'Z5*Z6'))
Htar_block.append((1, 'Z4*Z5'))
Henc_block.append((1, 'Z9*Z10'))
Htar_block.append((1, 'Z1*Z2'))
Henc_block += [(1, f'Z4*Z5'), ((8*lamb/3)**0.5, f'Z1*X6+Z3*X6')]
Htar_block.append((1, 'Z0*Z3'))
Henc_block += [(1, f'Z0*Z1'), ((8*lamb/3)**0.5, f'X2*Z5+X2*Z7')]
Htar = blocks2Mat(size//2, Htar_block)
Henc = blocks2Mat(size, Henc_block)

Hsim = lamb * Hpen + Henc
Heff_2nd_order = P0 @ Henc @ P0 - (P0 @ Henc @ Q0 @ HpenInverse @ Q0 @ Henc @ P0 / lamb)

Uenc = getU(4, blockNum)
Penc = Uenc @ Uenc.conj().T

print(checkSame(Penc @ Heff_2nd_order @ Penc, Uenc @ Htar @ Uenc.conj().T))
print(checkSame(Penc @ Heff_2nd_order @ (P0-Penc), np.zeros(Penc.shape)))

True
True


In [11]:
lamb = 1024
blockNum = 3
size = 4 * blockNum

Hpen_block = []
for i in range(blockNum):
    Hpen_block += [(1, f'X{4*i}*X{4*i+1}'), (3, f'Z{4*i}*Z{4*i+1}'), (1, f'X{4*i+2}*X{4*i+3}'), (3, f'Z{4*i+2}*Z{4*i+3}')]
Hpen = blocks2Mat(size, Hpen_block)
HpenInverse = np.linalg.pinv(Hpen)
Q0 = HpenInverse @ Hpen
P0 = np.identity(Q0.shape[0]) - Q0

Htar_block = []
Henc_block = []

Htar_block.append((1, 'Z0*Z1'))
Henc_block.append((1, 'Z1*Z2'))

Htar_block.append((1, 'Z2*Z3'))
Henc_block.append((1, 'Z5*Z6'))

Htar_block.append((1, 'Z4*Z5'))
Henc_block.append((1, 'Z9*Z10'))

Htar_block.append((1, 'Z1*Z2'))
Henc_block += [(1, f'Z4*Z5'), ((8*lamb/3)**0.5, f'Z1*X6+Z3*X6')]

Htar_block.append((1, 'Z0*Z3'))
Henc_block += [(1, f'Z0*Z1'), ((8*lamb/3)**0.5, f'X2*Z5+X2*Z7')]

Htar_block.append((1, 'Z3*Z4'))
Henc_block += [(1, f'Z8*Z9'), ((8*lamb/3)**0.5, f'Z5*X10+Z7*X10')]

Htar_block.append((1, 'Z2*Z5'))
Henc_block += [(1, f'Z4*Z5'), ((8*lamb/3)**0.5, f'X6*Z9+X6*Z11')]

Htar = blocks2Mat(size//2, Htar_block)
Henc = blocks2Mat(size, Henc_block)

Hsim = lamb * Hpen + Henc
Heff_2nd_order = P0 @ Henc @ P0 - (P0 @ Henc @ Q0 @ HpenInverse @ Q0 @ Henc @ P0 / lamb)

Uenc = getU(4, blockNum)
Penc = Uenc @ Uenc.conj().T

print(checkSame(Penc @ Heff_2nd_order @ Penc, Uenc @ Htar @ Uenc.conj().T))
print(checkSame(Penc @ Heff_2nd_order @ (P0-Penc), np.zeros(Penc.shape)))

True
False
